<a href="https://colab.research.google.com/github/frechezz/frechezz.github.io/blob/main/2ch_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title  { display-mode: "form" }
#@markdown <img height="22px" src="https://files.catbox.moe/im26b0.svg"><br><small>чтобы остановить выполнение кода в виджетовых ячейках: `Ctrl+M+I` или в меню: `Среда выполнения → Прервать выполнение кода`
'''
в сундуке — заяц, в зайце — утка,
в утке — яйцо, в яйце — игла

ячейка разделена на вложенные файлики
для ускорения запуска (не выполнения, а запуска, из-за построчной проверки кода гуглом).
в противном случае - холодный старт от 1,5 минуты
'''

# сброс сохраненных переменных на диске
%store -z
%cd /content
# задать чекпоинты автоматика (прошлый новый: ea9bd9fc7409109adcd61b897abc2c8881161256)
commit_new =  "0cc0ee1bcb4c24a8c9715f66cede06601bfc00c8"
commit_stable = "52f6e94338f31c286361802b08ee5210b8244141"
%store commit_new
%store commit_stable

clear_out = True # очищать вывод? (False)
def clear():
  if clear_out:
    clear_output()

import os
import os.path
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

# проверка рантайма и завершение, если лимит GPU исчерпан на аккаунте
nsmi = getoutput('nvidia-smi')
if "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in nsmi:
  display(HTML('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/warn'))
  !sleep 5
  print("завершение...")
  !sleep 5
  from google.colab import runtime
  runtime.unassign()

!rm /content/.config/*.ipynb >/dev/null 2>&1

# файл с переменными
# просмотр/редактирование: https://is.gd/1var_ipynb
!wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/1_var.ipynb -P /content/.config
%run /content/.config/1_var.ipynb


# загрузчик моделей
# просмотр/редактирование: https://is.gd/21models_ipynb
# файл с моделями: https://is.gd/22models_ipynb
!wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/2_1_models.ipynb -P /content/.config
%run /content/.config/2_1_models.ipynb


# создание виджетов для установки
# просмотр/редактирование: https://is.gd/3installation_ipynb
!wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/3_installation.ipynb -P /content/.config
%run /content/.config/3_installation.ipynb



# пикрограммы для допмоделей
extranets = HTML(
'''
<style>
@font-face {
  font-family: 'Models';
  src: url('https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/Models.woff2');
  src: url('https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/Models.otf');
}
span[title]:before {
  /*i - инпеинт, c - контролнет, p - пикс2пикс */
  font-family: "Models" !important;
  padding: 0 4px 0 0;
  font-size:2em !important;
  position: relative;
  top: 3px;
}
span[title="SD_v2_1"]:before,
span[title="A_to_Zovya_RPG_Artists_Tools"]:before,
span[title="img2img_PaintingGenerator"]:before,
span[title="Clarity"]:before,
span[title="SD_v1_5"]:before,
span[title="AIroticArtsPenis"]:before {
    content: "i";
}
span[title="Liberty"]:before {
    content: "ip";
  }

</style>
    '''
)

#начальное состояние доп.опций и действия по клику на них
gfolder.layout.display = 'none'
gdrive_models.layout.display = 'none'
gdrive_only.layout.display = 'none'
gdrive_only.layout.display = 'none'
gdrive_models.observe(gd_path, names='value')
gdrive.observe(gdrive_mount, names='value')
gdrive_only.observe(only_gdrive, names='value')
close_button.on_click(on_close_button_click)
#download_button.on_click(on_download_click) - кнопка скачать не нужна, все на кнопке продолжить
apply.on_click(apply_button_click)
# очистить вывод перед показом виджетов
clear()
# пауза 3-10сек (чтобы после очистки корректно отработал css)
!sleep 3
# отображение элементов виджетов в порядке сверху вниз
display(m)
display(HTML('<h2 id="setup_header">настройки</h2>'))
display(checkpoint)
display(gd_set, additions, webuiargs)
display(model_header)
display(search_and_button, tab)
display(filtered_checkboxes, close_button)
display(apply)
#скрыть кнопку "скачать" в виджете моделей
display(HTML('<style>.dl_button {display: none}</style>'))
#внедрение css
setup_css()
display(extranets)
cou.eval_js(f'new Audio("{gh}start_widget.mp3").play()') #оповещение о загрузке виджета

In [ ]:
#@title докачать модели{ display-mode: "form" }
#@markdown <a href="https://teletype.in/@stablediffusion/models_for_colab"><img height="24px" src="https://files.catbox.moe/wq11dz.svg"></a> <br>
#@markdown не нашел нужную модель? Модели/Lora/гиперсети/внедрения <a href="#scrollTo=TuwHDqutjgZd">можно добавить по прямым ссылкам

%store -r
%cd /content/stable-diffusion-webui/models/Stable-diffusion

clear_out = True # очищать вывод? (False)
def clear():
  if clear_out:
    clear_output()

import os.path
from google.colab import output as cou
from IPython.display import HTML, display
import ipywidgets as widgets
from IPython.display import clear_output
import requests
# папки указаны в исполняемом файле
!cd /content/stable-diffusion-webui/models/Stable-diffusion
cou.clear()
try:
  print(ok)
except NameError:
  # файл с переменными
  # просмотр/редактирование: https://is.gd/1var_ipynb
  !wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/1_var.ipynb -P /content/.config
  %run /content/.config/1_var.ipynb
  cou.clear()

# загрузчик моделей
# просмотр/редактирование: https://is.gd/21models_ipynb
# файл с моделями: https://is.gd/22models_ipynb
!wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/2_1_models.ipynb -P /content/.config
%run /content/.config/2_1_models.ipynb


# пикрограммы для допмоделей
extranets = HTML(
'''
<style>
@font-face {
  font-family: 'Models';
  src: url('https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/Models.woff2');
  src: url('https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/Models.otf');
}
span[title]:before {
  /*i - инпеинт, c - контролнет, p - пикс2пикс */
  font-family: "Models" !important;
  padding: 0 4px 0 0;
  font-size:2em !important;
  position: relative;
  top: 3px;
}
span[title="SD_v2_1"]:before,
span[title="A_to_Zovya_RPG_Artists_Tools"]:before,
span[title="img2img_PaintingGenerator"]:before,
span[title="Clarity"]:before,
span[title="SD_v1_5"]:before,
span[title="AIroticArtsPenis"]:before {
    content: "i";
}
span[title="Liberty"]:before {
    content: "ip";
  }

</style>
    '''
)

download_button.on_click(on_download_click)
close_button.on_click(on_close_button_click)
setup_css()
display(extranets)
!sleep 3
display(search_and_button, tab)
display(filtered_checkboxes, close_button)



---



In [ ]:
#@title запуск/перезапуск { display-mode: "form" }
#@markdown <sup>◀=|  для перезапуска webui нужно нажать сюда <br><small>чтобы остановить выполнение кода в виджетовых ячейках: `Ctrl+M+I` или в меню: `Среда выполнения → Прервать выполнение кода`

#@markdown [как составить промпт и список художников](https://teletype.in/@stablediffusion/artists_for_prompts) | [о настройках генерации](https://teletype.in/@stablediffusion/samplers_steps)
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

import os
import os.path


if (os.path.exists("/content/stable-diffusion-webui/ui-config.json") and os.path.exists("/content/stable-diffusion-webui/notification.mp3")):
  %store -r
  restart_header = widgets.HTML(value="<h2>Перезапуск установленного WebUI</h2>").add_class("restart_header")
  ngroktoken = widgets.Text(value="", placeholder="вставь свой токен ngrok сюда").add_class("ngroktoken")
  xformers_chk = widgets.Checkbox(value=True, description="xformers").add_class("xformers_chk")
  no_half_vae_chk = widgets.Checkbox(value=True, description="no-half-vae").add_class("no_half_vae_chk")
  restart = widgets.Button(description="🚀 презапуск webui").add_class("restart")
  chkbs = widgets.HBox([xformers_chk, no_half_vae_chk, ngroktoken, restart]).add_class("chkbs")

  restart_css = HTML("""
  <style>
  .widget-label {
    display: none!important;
  }
  .chkbs {
    display: flex;
    flex-wrap: wrap;
  }
  .chkbs input[type=checkbox] {
    width: 1em;
    height: 1em;
    padding: 0!important;
    margin: 0 0.5em 0 0!important;
    position: relative;
    top: 3px;
  }
  .xformers_chk,
  .no_half_vae_chk {
    width: auto!important;
    margin-right: 30px;
  }
  .chkbs span {
    font-size: 1.2em!important
  }
  .ngroktoken input,
  .ngroktoken input::placeholder {
    color: var(--colab-primary-text-color);
  }
  .restart {
    color: #f6f2e0;
    font-size: 1.2em;
    background: #2567b9;
    transition: 0.5s;
    width: auto!important;
    height: auto!important;
    padding: 0 18px 0 15px;
    margin-left: 20px;
  }
  .restart:hover {
    font-size: 1.22em;
    background: #1d5192;
    width: auto!important;
    height: auto!important;
  }
  </style>
  """)

  try:
    print(ok)
  except NameError:
    !wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/1_var.ipynb -P /content/.config
    %run /content/.config/1_var.ipynb

  %cd /content/stable-diffusion-webui
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  !sleep 3
  #проигрывание аудио для влияния на подсознание
  from google.colab import output as cou
  cou.eval_js("var audio = new Audio('https://raw.githubusercontent.com/PR0LAPSE/StableDiffusionWebUIColab/main/src/rectal_whisper.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")

  !find /content/ -name "*.safetensors" -type f -size +1900M -exec mv {} /content/stable-diffusion-webui44/models/Stable-diffusion/ \;
  !find /content/stable-diffusion-webui/ -name "*.safetensors" -type f -size +1900M -exec mv {} /content/stable-diffusion-webui44/models/Stable-diffusion/ \;
  clear_output()

  def restart_webui(b):
    %store -d xformers
    %store -d no_half_vae
    ngrok_token = ngroktoken.value
    xformers = xformers_chk.value
    no_half_vae = no_half_vae_chk.value

    clear_output()

    #Установка токена ngrok
    ngrok_share = cfg+'cfg'
    freetoken = open(ngrok_share).read().split('\n')
    sharetoken = random.choice(list(freetoken))
    ngrok_token_value = ngrok_token
    xformers_value = xformers
    no_half_vae_value = no_half_vae
    if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
      ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
    else:
      ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
    if xformers_value == True:
      xf = '--xformers'
    if xformers_value == False:
      xf = '--opt-split-attention'
    if no_half_vae_value == True:
      nhv = '--no-half-vae'
    if no_half_vae_value == False:
      nhv = ''
    if checkpoint_value == "new":
      gq = '--gradio-queue --disable-console-progressbars --no-hashing --opt-channelslast'
    if checkpoint_value == "legacy":
      gq = ''
    ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
    if booru_value == True:
      deepdanbooru_arg = "--deepdanbooru"
    if booru_value == False:
      deepdanbooru_arg = ""
    default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share {deepdanbooru_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
    cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
    #with capture.capture_output() as stored_variables:
    #  %store cmdargs
    saved_vars = open(f'{cfg}saved_vars', 'wb')
    pickle.dump(xf, saved_vars)
    pickle.dump(nhv, saved_vars)
    pickle.dump(gq, saved_vars)
    pickle.dump(deepdanbooru_arg, saved_vars)
    pickle.dump(default_args, saved_vars)
    saved_vars.close()
    print("\033[1;36mВнимание! Если интерфейсе не отображается генерация, просто используй ссылку на Gradio!\033[0m \n")
    print("\033[1;34mПереходить по ссылкам можно после полной прогрузки, ожидай сообщения.\033[0m")
    flara()
    with open('/content/stable-diffusion-webui/clfr.txt', 'r') as flare:
      print("\033[01;38;05;112m⯈\033[0m Ссылка на Флару: " + flare.read())
    #from google.colab.output import serve_kernel_port_as_window
    #from google.colab.output import eval_js
    #print("\033[01;38;05;112m⯈\033[0m Ссылка на Гугл:", eval_js( "google.colab.kernel.proxyPort(" + str(7860) + ")" ))
    print("")
    #Собственно запуск

    # в последней версии автоматик добавил аргументы: --no-hashing, можно использовать вместе с --lowram
    !python /content/stable-diffusion-webui/webui.py $cmdargs
    if os.path.exists("/content/gdrive/MyDrive/SD/styles.csv"):
      !cp -f /content/stable-diffusion-webui/styles.csv /content/gdrive/MyDrive/SD/styles.csv
    display(HTML('/content/.config/end'))

  restart.on_click(restart_webui)
  display(restart_css, restart_header)
  display(chkbs)
else:
  print('вебуи не установлен!\nсначала нужно выполнить установку в ячейке "настройки, модели, установка, запуск"')


<details><summary><a href="https://openuserjs.org/scripts/Kuroi_mato_O/fixed_TOC_for_colab"><font size=4>Юзерскрипт для быстрой навигации и автоматического переподключения</a></summary><img src="https://files.catbox.moe/qawiz8.png"></details> 
<br/>
<!--
<details> 
  <summary><font size="3">JS-коды для консоли браузера, которые будут автоматически нажимать кнопку повторного подключения при бездействии.</summary>
<p>Открывай консоль разработчика в браузере (Ctrl+Shift+J - хромиумподобные или Ctrl+Shift+K - фаерфоксоподобные):
<img src="https://telegra.ph/file/fc25b681ea1900e042919.png" />
код можно скопировать, заменить 60000 на 120 для проверки и нажать Enter, если увидишь что кнопка быстро нажимается - код рабочий, обновляй страницу и используй код без изменений:</p>
<pre>
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}

setInterval(ClickConnect, 60000)
</pre>
<pre>
function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
</pre>
<pre>
function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)
</pre>
<pre>
function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)
</pre>
<pre>
 function ClickConnect(){
   btn = document.querySelector("colab-toolbar-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
</pre>
К сожалению, напрямую из колаба уже не работают. Работают из консоли до сих пор нормально, а так же из них можно сделать user.js. Если перестали работать, значит гугл переименовал  id кнопки, зайди в исходный код страницы и подсмотри новый id, обнови id в скрипте.<br/>
</details> -->

<details><summary>быстрые ссылки</summary>
<ul>
<li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li>
<li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li>
<li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li>
<li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li>
<li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li>
<li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li>
<li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li>
<li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li>
<li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li>
<li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li>
<li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li>
<li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li>
<li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li>
</ul></details>



---

In [ ]:
#@title 💾 Создание и скачивание архива со всеми картинками { display-mode: "form" }
#@markdown <details><summary><font color='#c89c26' size='3' face="Verdana"><b>👈 Прочитать описание</b></font></summary><strong>Cкачивание напрямую на ПК, минуя гуглодиск или получение ссылки с архивом всех картинок</strong>: это очень полезно, если ты не монтируешь гуглодиск в colab. Имей в виду, что прямое скачивание, которе тут предлагается, средствами colab - оно очень медленное, а самое печальное - если файлик не маленький, то загрузка вообще будет прерываться, вызывая боль и мучения. Поэтому в качестве альтернативы - воспользуйся очень быстрой выгрузкой архива на anonfiles или его саттелиты по своему вкусу, потом просто гораздо быстрее и без нервов скачаешь.<br><strong>Прежде чем запустить скачивание архива в этой ячейке, ячейка с WebUI должна быть остановлена.</strong> (потом можно будет ее запустить снова, если требуется)</details>
что_сделать_после_архивирования = "\u0432\u044B\u0433\u0440\u0443\u0437\u0438\u0442\u044C \u0438 \u043F\u043E\u043B\u0443\u0447\u0438\u0442\u044C \u0441\u0441\u044B\u043B\u043A\u0443" #@param ["сразу скачать", "выгрузить и получить ссылку"]
файлопомойка = "anonfiles.com" #@param ["anonfiles.com", "openload.cc", "bayfiles.com", "lolabits.se", "vshare.is", "upvid.cc", "hotfile.io", "rapidshare.nu", "filechan.org", "letsupload.cc", "share-online.is", "megaupload.nz", "myfile.is"]
import datetime
from datetime import datetime
from datetime import date
import pytz
from pytz import timezone
now = datetime.now(pytz.timezone('Europe/Moscow'))
filename = "SD_" + now.strftime("%d%m%Y_%H%M%S") + ".zip"
!7z a -tzip -mx=4 "{filename}" /content/output
output.clear()
if что_сделать_после_архивирования == "сразу скачать":
  print('скачивание архива начнется автоматически...\nбраузер может заблокировать загрузку, не забудь выдать разрешения')
  from google.colab import files
  files.download(filename)
if что_сделать_после_архивирования == "выгрузить и получить ссылку":
  import requests
  url = f'https://api.{файлопомойка}/upload'
  x = requests.post(url, files = {'file': open(filename,'rb')},)
  print("\033[01;38;05;46m Твоя ссылка: " + x.json()["data"]["file"]["url"]["full"])

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title 🆑 удалить и сбросить среду выполнения { display-mode: "form" }
#@markdown <sup>◀=| нажать кнопку самоуничтожения
from google.colab import runtime
runtime.unassign()

In [ ]:
#@title 📁 Файловый менеджер { display-mode: "form" }
#@markdown <sup>◀=| открыть более удобный менеджер файлов в новой вкладке
import os
import os.path
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/imjoy_elfinder"):
  !pip -q install imjoy-elfinder
from IPython.display import HTML
import threading
from google.colab import output
from imjoy_elfinder.app import main
from IPython.display import clear_output
from google.colab.output import serve_kernel_port_as_window
from google.colab.output import eval_js
output.clear()
threading.Thread(target=main, args=[["--root-dir=/content", "--port=8765"]]).start()
#output.serve_kernel_port_as_iframe(8765, height='620')
#google.colab.serve_kernel_port_as_window(8765)
!sleep 1
output.clear()
flink = eval_js( "google.colab.kernel.proxyPort(" + str(8765) + ")" )
display(HTML(f'<a href="{flink}" target="_blank" style="text-decoration:none;"><h2 style="color: #c06f0e;">📁ссылка на файловый менеджер<h2></a>'))

In [ ]:
#@title <img src='https://files.catbox.moe/3uju7d.svg' height='30px'> фикс утечки памяти | перезапуск среды выполнения { display-mode: "form" }
#@markdown <sup>◀=| нажать чтобы сбросить память или перезапустить рантайм
import os
import os.path
if os.path.exists("/content/stable-diffusion-webui"):
  print("\033[1;31mВНИМАНИЕ!\nПосле выполнения этой ячейки произойдет перезапуск среды выполнения!\n\033[1;34m")
else:
  print("\033[1;36mПосле выполнения этой ячейки произойдет перезапуск среды выполнения.")

#!for pid in $(ps -ef | grep -v "awk" | awk '/ipykernel_launcher/ {print $2}'); do kill -9 $pid; done
!sleep 5
!kill $(ps aux | sort -rnk 4 | awk 'NR>1{print $2; exit}')
exit()

In [ ]:
#@title <img height="20px" src="https://kstatic.googleusercontent.com/files/d57b24106c34c7e50ef3d98423b94ddaf35ad2da73a9b9d4d12f52dbb9dd4c08c2957f6255ab8690d5ef0b32cff8287e09577d05e479d263e872160c4c9e8363"> перенос внутри гуглодиска моделей СД, внедрений, гиперсетей, НиРа, ВАЭ, моделей апскейлеров и т.п. в правильные папки { display-mode: "form" }
#@markdown <sup>◀=| открыть виджет реорганизации файлов </sup><br><details><summary>как это должно выглядеть</summary><img width="350px" src="https://files.catbox.moe/sdm9mi.png"></details>
!rm /content/.config/*.ipynb >/dev/null 2>&1
!wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/gdrive_folders.ipynb -P /content/.config/
%run /content/.config/gdrive_folders.ipynb



---



In [ ]:
#@title Переподключение, сброс кэша гуглодиска { display-mode: "form" }
#@markdown <center><img src='https://kstatic.googleusercontent.com/files/d57b24106c34c7e50ef3d98423b94ddaf35ad2da73a9b9d4d12f52dbb9dd4c08c2957f6255ab8690d5ef0b32cff8287e09577d05e479d263e872160c4c9e8363' /></center>
#@markdown Гуглодиск монтируется по пути: /content/gdrive/MyDrive<br>Если к аккаунту подключен "общий диск", он находится в папке: /content/gdrive/Shareddrives (например: <n>/con<n>tent</n>/<n>drive</n>/<n>Share</n><n>ddrives</n>/<n>Vasya</n>)
режим = "\u043F\u0440\u0438\u043C\u043E\u043D\u0442\u0438\u0440\u043E\u0432\u0430\u0442\u044C" #@param ["примонтировать", "размонтировать"]

from google.colab import drive
drive.mount._DEBUG = False
if режим == "примонтировать":
  drive.mount('/content/drive', force_remount=True)
  print('гуглодиск примонтирован')
elif режим == "размонтировать":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  !rm -rf /root/.config/Google/DriveFS
  print('гуглодиск размонтирован')
#@markdown Можно примонтировать гуглодиск, если забыл сделать это ранее. Размонтирование необходимо в случае проблем с гуглодиском (в гуглодиске не появляются отправленные с колаба файлы и т.п.)<br>Подключение гуглодиска в этой ячейке не меняет настройки сохранения картинок и папку моделей!
#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>



---





---



#👈 Загрузка моделей с сайтов и торрентов

In [ ]:
#@title  массовая загрузка моделей и т.п. по прямым ссылкам { vertical-output: true, display-mode: "form" }

#@markdown <sup>◀=| запусти ячейку чтобы указать ссылки

#@markdown Скачать: <a href="https://rentry.org/mato_loras">Lora отобранные Аноном</a> | <a href="https://civitai.com">Модели и прочие экстрасети</a> | <a href="https://huggingface.co/models?pipeline_tag=text-to-image&sort=modified">Модели</a>

import os
import os.path
from IPython.display import HTML, Javascript, display, clear_output
import ipywidgets as widgets
urls_list = widgets.Textarea(value='', placeholder='вставь каждую ссылку в новую строку\nвсе ссылки должны быть только прямыми непосредственно на загрузку файла\nпримеры ссылок:\nhttps://civitai.com/api/download/models/4194\nhttps://huggingface.co/wavymulder/portraitplus/resolve/main/portrait%2B1.0.safetensors').add_class("urls_textarea")
where = widgets.Dropdown(options=[('SD-модели', 1), ('лоры', 2), ('внедрения', 3), ('гиперсети', 4), ('произвольный путь', 5)], value=1, description='куда скачивать:').add_class("where")
dlbtn = widgets.Button(description='скачать').add_class("dlbtn")
ownpath = widgets.Text(value='', placeholder='вставь путь до папки').add_class("ownpath")
wcss = HTML('''
<style>
.urls_textarea textarea {
  background-color: transparent;
  background-image: url(https://files.catbox.moe/4e5x85.png);
  background-position: -5px -4px;
  background-repeat: no-repeat;
  background-size: 35px auto;
  padding: 1px 30px;
  width:100%!important;
  min-height: 40px!important;
  max-height:1830px!important;
  height:550px!important;
  font-size: 14px;
  line-height: 30px;
  height: 460px!important;
  font-family: Consolas, monospace!important;
}
.urls_textarea  {
  width: 45%!important;
  max-width: 1200px!important;
  min-width: 800px!important;
  height: 460px!important;
  
}
.back {
  position: absolute;
  left: 6px;
  top: 9px;
  right: 20px;
  bottom: 11px;
  background: repeating-linear-gradient(to bottom, #303030, #161616 2px, #161616 30px, #161616 2px); 
  border-top: 30px #161616 solid;
  z-index: -1;
  width: 44.8%!important;
  max-width: 1200px!important;
  min-width: 800px!important;
  height: 428px!important;
 }
.where, .where label {
	  width: auto!important
}
.where, .dlbtn, .ownpath {
   	margin-top: 15px;
    margin-bottom: 15px;
    margin-right: 10px;
 }
.dlbtn {
	background: #22b234;
	color: #fff;
	font-size: 18px;
	transition: 0.2s;	
}
.dlbtn:hover {
	transform: scale(105%);
	background: #27cd3b;
	color: #1d1d1d;
}
</style>
<div class="back"></div>
''')

def dl_ownpath(change):
  if where.value == 5:
    ownpath.layout.display = 'flex'
  else:
    ownpath.layout.display = 'none'
ownpath.layout.display = 'none'
where.observe(dl_ownpath, names='value')
def dlbtn_click(b):
  if where.value == 1:
    pth = "/content/stable-diffusion-webui/models/Stable-diffusion"
  if where.value == 2:
    pth = "/content/stable-diffusion-webui/models/Lora"
  if where.value == 3:
    pth = "/content/stable-diffusion-webui/embeddings"
  if where.value == 4:
    pth = "/content/stable-diffusion-webui/models/hypernetworks"
  if where.value == 5:
    pth = ownpath.value
  with open("/content/.config/urls","w") as urls:
    urls.write(urls_list.value)
  clear_output()
  !cat /content/.config/urls | xargs -P 100 -n1 wget -P {pth} -nv -t 10 --show-progress --progress=bar -q --content-disposition
  if where.value == 2:
    !ln -s /content/stable-diffusion-webui/models/Lora/* /content/stable-diffusion-webui/extensions/sd-webui-additional-networks-dev2/models/lora 2>/dev/null
    !ln -s /content/stable-diffusion-webui/models/Lora/* /content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora 2>/dev/null
  !rm /content/.config/urls
  
dlbtn.on_click(dlbtn_click)
display(wcss)
display(urls_list)
display(widgets.HBox([where, ownpath, dlbtn]))

In [ ]:
#@title <big>libtorrent</big> { display-mode: "form" }

import os
import time
from google.colab import output
if подключить_гуглодиск:
  from google.colab import output
  from google.colab import drive
  drive.mount('/content/gdrive')
  !sleep 2
if not os.path.exists("/usr/share/doc/python3-libtorrent/copyright"):
  print ("сначала нужно установить торрент-клиент.\n установка...")
  !python -m pip install jedi>=0.10 --quiet
  !python -m pip install --upgrade pip setuptools wheel --quiet
  !python -m pip install lbry-libtorrent --quiet
  !apt install -qq python3-libtorrent --quiet
  !rm '/content/=0.10'
  output.clear()
  print("установка клиента завершена")
else:
  !sleep 1

import ipywidgets as widgets
import libtorrent as lt
from threading import Thread

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

!sleep 1
#@markdown чтобы скачать на гуглодиск, сначала примонтируй его, если этого не сделал ранее, или укажи другую папку для загрузки
подключить_гуглодиск = False #@param {type:"boolean"}
куда_качать = "/content/gdrive/My Drive/Torrent" #@param {type:"string"}
#@markdown Поставь галочку, если нужно после запуска выбрать файлы .torrent с компьютера:
torrent_файл = False #@param {type:"boolean"}
#@markdown Или вставь magnet-ссылку:


# выбрать торрент файлы?
if torrent_файл:
  from google.colab import files

  source = files.upload()
  if not source:                         
    print ('Нечего скачивать...')
    print ('Ты либо загрузи torrent-файлы, либо сними галочку и укажи magnet-ссылку')
    print ('попробуй еще раз')
    exit
  else:
    params = {
        "save_path": f"{куда_качать}",
        "ti": lt.torrent_info(list(source.keys())[0]),
    }
    downloads.append(ses.add_torrent(params))
    print ('ГОТОВО !!!')
else:
  from numpy import empty
  import libtorrent as lt
  import time
  import datetime
  ses = lt.session()
  ses.listen_on(6881, 6891)
  params = {
      'save_path': f'{куда_качать}',
      }
  link = "magnet:?xt=urn:btih:ED295B74E820777FF1E5C84B6657CB2AEB539AE1&tr=http%3A%2F%2Fbt3.t-ru.org%2Fann%3Fmagnet&dn=%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%20%D0%91%D0%B5%D0%B9%D0%BB%D1%8C%D0%BA%D0%B8%D0%BD%20-%20%D0%A1%D0%B5%D1%80%D0%B8%D1%8F%20%D0%BA%D0%BD%D0%B8%D0%B3%20%22%D0%A1%D0%B5%D0%BA%D1%81-%D0%BB%D0%B8%D0%BA%D0%B1%D0%B5%D0%B7%22%20%D0%BF%D0%BE%20%D0%B3%D0%BE%D0%BC%D0%BE%D1%81%D0%B5%D0%BA%D1%81%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8%20%D0%B8%20%D1%81%D0%B5%D0%BA%D1%81%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8%20%5B2006-2009%2C%20DOC%5D" #@param {type:"string"}

  if not link:                         
      print ('Ты не ввел магнет-ссылку !!!')
      exit
  if "magnet:?xt=urn:btih" not in link:
    print ('Это не магнет-ссылка !!!')
    print ('Либо вставь нормальную магнет-ссылку, либо отметь галочку и запусти и загрузи торрент-файлы.')
    exit
  else:
    handle = lt.add_magnet_uri(ses, link, params)
    ses.start_dht()

    begin = time.time()
    print(datetime.datetime.now())

    print ('Получаю метаданные...')
    while (not handle.has_metadata()):
      time.sleep(1)
    print ('Запускаю загрузку...')

    while (handle.status().state != lt.torrent_status.seeding):
        s = handle.status()
        state_str = ['очередь', 'проверка', 'получение метаданных', \
                'идет скачивание', 'готово', 'раздача', 'распределение']
        print ('%.2f%% (загрузка: %.1f kb/s отдача: %.1f kB/s пиры: %d) %s ' % \
                (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
                s.num_peers, state_str[s.state]))
        time.sleep(5)

    end = time.time()
    print("ГОТОВО!")

    print("Затрачено времени: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")


#@markdown Обрати внимание: путь до папки загрузок приведен для примера (если подключен гуглодиск, допустим)! Это не путь до папки с моделями в колабе!

#@markdown Путь до стандартной папки с моделями: `/content/stable-diffusion-webui/models/Stable-diffusion`
#@markdown Торент может содержать вложенные папки и т.п., поэтому скачать лучше куда-то, а потом непосредственно файл можели перетянуть в папку где лежат все модели:
#@markdown <b><font color='forestgreen' size='2'><details><summary>скрин где найти тут папку</summary> <img src='https://files.catbox.moe/4pi0d7.png' /></details>


#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

# 👈 Прунинг и конвертация моделей

In [ ]:
#@title Проверка смещения тензоров модели { display-mode: "form" }
модель = "/content/aloeVeraSSimpMaker3K_simpMaker3K1-ema-fp16.safetensors" #@param {type:"string"}
import torch
import safetensors.torch
import os.path
import sys
if __name__ == "__main__":
    checkpoint_file = модель
    device = 'cpu'
    ext = os.path.splitext(checkpoint_file)

    if ext[-1] == ".safetensors":
        checkpoint = safetensors.torch.load_file(checkpoint_file, device=device)
    elif ext[-1] == ".ckpt":
        checkpoint = torch.load(checkpoint_file,  map_location=torch.device(device))
    else:
        raise Exception("файл модели должен быть с расширением ckpt или safetensors")

    if 'state_dict' in checkpoint:
        checkpoint = checkpoint['state_dict']

    print(checkpoint['cond_stage_model.transformer.text_model.embeddings.position_ids'])

In [ ]:
#@title Прунинг моделей (с поддержкой v2)
import os
from os import path
from pathlib import Path
import enum
from google.colab import output
if not os.path.exists("/content/stable-diffusion-prune/prune.py"):
  from google.colab import output
  %cd /content/
  !git clone https://github.com/saftle/stable-diffusion-prune
  output.clear()
  print("Установка зависимостей...")
  #если ModuleNotFoundError:
    #!pip -q install -r https://github.com/AUTOMATIC1111/stable-diffusion-webui/raw/master/requirements.txt
  output.clear()
#@markdown Сперва нужно указать путь до исходной модели (для пруна одиночного файла) или папки с моделями (для пакетной обработки)
источник = "/content/stable-diffusion-webui/models/Stable-diffusion/theGreatFault_v11.ckpt" #@param {type:"string"}
добавлять_суффиксы_к_имени = True #@param {type:"boolean"}
удалить_исходник = True #@param {type:"boolean"}
сконвертировать_в_safetensors = True #@param {type:"boolean"}
#@markdown Галочками нужно отметить параметры прунинга:<br>
#@markdown `fp16 - конвертировать в float16` <br> `ema - оставить только EMA данные` <br> `no_clip - удалить веса CLIP` <br> `no_vae - удалить веса VAE` <br> `no_depth - удалить данные глубины (для моделей v2)` <br> `no_unet - удалить веса UNet`
fp16 = True #@param {type:"boolean"}
ema = False #@param {type:"boolean"}
no_clip = False #@param {type:"boolean"}
no_vae = False #@param {type:"boolean"}
no_depth = False #@param {type:"boolean"}
no_unet = False #@param {type:"boolean"}

if сконвертировать_в_safetensors:
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/safetensors"):
    !pip -q install safetensors
    output.clear()
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pytorch_lightning"):
    !pip -q install pytorch-lightning
    output.clear()
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/torchsde"):
    !pip -q install torchsde
    output.clear()
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pandas"):
    !pip -q install pandas
    output.clear()

if fp16:
  flpnt16 = "--fp16"
  f16 = "_fp16"
if not fp16:
  flpnt16 = ""
  f16 = ""

if ema:
  exmovaver = "--ema"
  em = "_ema"
if not ema:
  exmovaver = ""
  em = ""

if no_clip:
  clip = "--no-clip"
  clp = "_no-clip"
if not no_clip:
  clip = ""
  clp = ""

if no_vae:
  vae = "--no-vae"
  nv = "_no-vae"
if not no_vae:
  vae = ""
  nv = ""

if no_depth:
  depth = "--no-depth"
  nd = "_no-depth"
if not no_depth:
  depth = ""
  nd = ""

if no_unet:
  unet = "--no-unet"
  nu = "_no-unet"
if not no_unet:
  unet = ""
  nu = ""
ckptpath = Path(источник)
folderpath = os.path.dirname(источник) + "/"
if добавлять_суффиксы_к_имени:
  suff = f16 + em + clp + nv + nd + nu
else:
  suff = "_temp"
extention = os.path.splitext(источник)[-1]
outputname = folderpath + ckptpath.stem + suff + extention
input_type = os.path.isfile(источник)
# Обработка одиночного файла
if input_type:
  print("\033[01;38;05;136mПрунинг модели...")
  !python /content/stable-diffusion-prune/prune.py {flpnt16} {exmovaver} {clip} {vae} {depth} {unet} "{источник}" "{outputname}"
  print("\033[01;38;05;34mГотово!")
  class SIZE_UNIT(enum.Enum):
    BYTES = 1
    KB = 2
    MB = 3
    GB = 4
  def convert_size(size_in_bytes, unit):
    if unit == SIZE_UNIT.KB:
        return size_in_bytes/1024
    elif unit == SIZE_UNIT.MB:
        return size_in_bytes/(1024*1024)
    elif unit == SIZE_UNIT.GB:
        return size_in_bytes/(1024*1024*1024)
    else:
        return size_in_bytes
  def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
    size = os.path.getsize(file_name)
    return convert_size(size, size_type)
  size_in = get_file_size(источник, SIZE_UNIT.GB)
  size_out = get_file_size(outputname, SIZE_UNIT.GB)
  print('Размер новой модели:', round(size_out, 2) ,  'Гб!\nА размер исходной:  ', round(size_in, 2),  'Гб!')
  if удалить_исходник:
    !rm "{источник}"
  if not добавлять_суффиксы_к_имени:
    rename = outputname.replace("_temp", "")
    os.rename(outputname, rename)
    outputname = rename
  if сконвертировать_в_safetensors:
    sftname = path.splitext(outputname)[0]
    import torch
    from safetensors.torch import save_file
    try:
      weights = torch.load(f"{outputname}")["state_dict"]
      save_file(weights, f"{sftname}.safetensors")
      !rm {outputname}
    except RuntimeError:
      print("сконвертировать на GPU не получилось!\nконвертирую на CPU")
      weights = torch.load(f"{исходная_ckpt}", map_location=torch.device('cpu'))["state_dict"]
      save_file(weights, f"{sfname}.safetensors")
      !rm {outputname}
    except ValueError:
      print(f'\033[38;05;143mМодель {f} содержит код NovelAI, колаб возможно крашнется из-за нехватки памяти!\033[0m')
      with torch.no_grad():
        weights = torch.load(f"{outputname}")["state_dict"]
        weights.pop("state_dict")
        sft = f"{outputname.replace('.ckpt', '')}.safetensors"
        print(f'Пытаюсь сохранить {sft}...')
        save_file(weights, sft)
        print(f'Успешно сохранил {sft}')
        !rm {outputname}
    except KeyError:
      with torch.no_grad():
        weights = torch.load(f"{outputname}")
        sft = f"{outputname.replace('.ckpt', '')}.safetensors"
        print(f'Пытаюсь сохранить {sft}...')
        save_file(weights, sft)
        print(f'Успешно сохранил {sft}')
        !rm {outputname}
    except Exception as er:
      print(f'\033[01;38;05;167mОшибка конвертации: {er}\033[0m')
# Обработка папки   
if not input_type:
  print("пакетный прунинг")
  ckptfiles = os.listdir(источник)
  for f in ckptfiles:
    if f.lower().endswith('.ckpt'):
      inputckpt = источник + "/" + f
      ext = os.path.splitext(inputckpt)[-1]
      outckpt = источник + "/" + os.path.splitext(f)[0] + suff + ext
      print(f"\033[01;38;05;136mВыполняется сжатие: \033[01;38;05;38m{f}\033[m")
      !python /content/stable-diffusion-prune/prune.py {flpnt16} {exmovaver} {clip} {vae} {depth} {unet} "{inputckpt}" "{outckpt}"
      class SIZE_UNIT(enum.Enum):
        BYTES = 1
        KB = 2
        MB = 3
        GB = 4
      def convert_size(size_in_bytes, unit):
        if unit == SIZE_UNIT.KB:
            return size_in_bytes/1024
        elif unit == SIZE_UNIT.MB:
            return size_in_bytes/(1024*1024)
        elif unit == SIZE_UNIT.GB:
            return size_in_bytes/(1024*1024*1024)
        else:
            return size_in_bytes
      def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
        size = os.path.getsize(file_name)
        return convert_size(size, size_type)
      size_in = get_file_size(inputckpt, SIZE_UNIT.GB)
      size_out = get_file_size(outckpt, SIZE_UNIT.GB)
      print('Размер новой модели:', round(size_out, 2) ,  'Гб!\nА размер исходной:  ', round(size_in, 2),  'Гб!')
      if удалить_исходник:
        !rm {inputckpt}
      if not добавлять_суффиксы_к_имени:
        renamemodel = outckpt.replace("_temp", "")
        os.rename(outckpt, renamemodel)
        outckpt = renamemodel
      if сконвертировать_в_safetensors:
        sftname = path.splitext(outckpt)[0]
        import torch
        from safetensors.torch import save_file
        try:
          weights = torch.load(f"{outckpt}")["state_dict"]
          save_file(weights, f"{sftname}.safetensors")
          !rm {outckpt}
        except RuntimeError:
          print("сконвертировать на GPU не получилось!\nконвертирую на CPU")
          weights = torch.load(f"{исходная_ckpt}", map_location=torch.device('cpu'))["state_dict"]
          save_file(weights, f"{sfname}.safetensors")
          !rm {outckpt}
        except ValueError:
          print(f'\033[38;05;143mМодель {f} содержит код NovelAI, колаб возможно крашнется из-за нехватки памяти!\033[0m')
          with torch.no_grad():
            weights = torch.load(f"{outckpt}")["state_dict"]
            weights.pop("state_dict")
            sft = f"{outckpt.replace('.ckpt', '')}.safetensors"
            print(f'Пытаюсь сохранить {sft}...')
            save_file(weights, sft)
            print(f'Успешно сохранил {sft}')
            !rm {outckpt}
        except KeyError:
          with torch.no_grad():
            weights = torch.load(f"{outckpt}")
            sft = f"{outckpt.replace('.ckpt', '')}.safetensors"
            print(f'Пытаюсь сохранить {sft}...')
            save_file(weights, sft)
            print(f'Успешно сохранил {sft}')
            !rm {outckpt}
        except Exception as er:
          print(f'\033[01;38;05;167mОшибка конвертации: {er}\033[0m')

        print("\033[01;38;05;34mМодель сохранена в safetensors")
      continue
  print("---------------------------\n\033[01;38;05;34mВсё!")

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Альтернативный прунер, если с другими не получилось { display-mode: "form" }
моделька = "/content/new/Radiant_v3.ckpt" #@param {type:"string"}
import os
from google.colab import output
if not os.path.exists("/content/pruner/prune-ckpt.py"):
  !wget -nv https://github.com/pesser/stable-diffusion/raw/main/scripts/prune-ckpt.py -P /content/pruner
try:
    import fire 
except ImportError:
    !pip -q install fire
try:
    import torch-fidelity
except ImportError:
    !pip -q torch-fidelity
try:
    import einops 
except ImportError:
    !pip -q install einops
output.clear()
try:
  !python /content/pruner/prune-ckpt.py {моделька} --keep_only_ema
except KeyError:
  print("не получилось, попробуем другой метод...")
  if not os.path.exists("/content/pruner/prune_ckpt.py"):
    !wget -nv https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/prune_ckpt.py -P /content/pruner
    !python /content/pruner/prune_ckpt.py {моделька} --keep_only_ema
    print("УСПЕХ!")
except Exception as er:
  print(f"Ошибка: {er} \nВозможно стоит попробовать метод через слияние модели самой с собой с 0 смешиванием через ВебУИ Автоматик111 с опцией fp16")
#@markdown Некоторые заковыристые миксы, которые содержат хрен пойми какие модели, в том числе с кодом из НовелАИ, по нескольку раз перемиксованные, не получается отпрунить стандартными методами. Этот вариант может иногда помочь (сбрасывает состояние оптимизатора и не-ЕМА веса), после него полученую модель нужно будет пропустить через обычный прунер, для получения fp16 и safensor.

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Конвертор 🥒ckpt в 🤗safetensors (пакетный и одиночный режим)
папка_ckpt = "/content/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
import os
from os import path
from google.colab import output
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/safetensors"):
  !pip -q install safetensors
  output.clear()
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pytorch_lightning"):
  !pip -q install pytorch-lightning
  output.clear()
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/torchsde"):
  !pip -q install torchsde
  output.clear()
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pandas"):
  !pip -q install pandas
  output.clear()
import torch
from safetensors.torch import save_file
%cd {папка_ckpt}
files = os.listdir()
for f in files:
  if f.lower().endswith('.ckpt'):
    print(f'Загружаю {f}...')
    fn = f"{f.replace('.ckpt', '')}.safetensors"
    if fn in files:
      print(f'\033[38;05;136mПропуск, поскольку {fn} уже существует.\033[0m')
      continue
    if "novelai" in fn:
      try:
        with torch.no_grad():
          weights = torch.load(f)["state_dict"]
          weights.pop("state_dict")
          fn = f"{f.replace('.ckpt', '')}.safetensors"
          print(f'Пытаюсь сохранить {fn}...')
          save_file(weights, fn)
          print(f'Успешно сохранил {fn}')
          continue
      except RuntimeError:
        with torch.no_grad():
          weights = torch.load(f, map_location=torch.device('cpu'))["state_dict"]
          weights.pop("state_dict")
          fn = f"{f.replace('.ckpt', '')}.safetensors"
          print(f'Пытаюсь сохранить {fn}...')
          save_file(weights, fn)
          print(f'Успешно сохранил {fn}')
          continue
    try:
      with torch.no_grad():
        weights = torch.load(f)["state_dict"]
        fn = f"{f.replace('.ckpt', '')}.safetensors"
        print(f'Сохраняю {fn}...')
        save_file(weights, fn)
    except EOFError:
      print(f'\033[01;38;05;167mФайл {f} поврежден, недокачан, или не является файлом модели!\033[0m')
    except KeyError:
      !sleep 3
      with torch.no_grad():
        weights = torch.load(f)
        fn = f"{f.replace('.ckpt', '')}.safetensors"
        print(f'Пытаюсь сохранить {fn}...')
        save_file(weights, fn)
    except ValueError:
      !sleep 3
      print(f'\033[38;05;143mМодель {f} содержит код NovelAI, сейчас колаб крашнется из-за нехватки памяти!\033[0m')
      with torch.no_grad():
        weights = torch.load(f)["state_dict"]
        weights.pop("state_dict")
        fn = f"{f.replace('.ckpt', '')}.safetensors"
        print(f'Пытаюсь сохранить {fn}...')
        save_file(weights, fn)
        print(f'Успешно сохранил {fn}')
    except RuntimeError:
      with torch.no_grad():
        weights = torch.load(f, map_location=torch.device('cpu'))["state_dict"]
        fn = f"{f.replace('.ckpt', '')}.safetensors"
        print(f'Пытаюсь сохранить {fn}...')
        save_file(weights, fn)
        print(f'Успешно сохранил {fn}')
    except Exception as ex:
      print(f'\033[01;38;05;167mОшибка конвертации {f}: {ex}\033[0m')
print('\033[38;05;34mГотово!\033[0m')
%cd /content/

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Конвертор 🥒ckpt в 🤗safetensors для базовых моделей (одиночный режим)
#@markdown После конвертации модель в новом формате будет лежать рядом с исходной в той же папке. [GUI-вариант](https://github.com/diStyApps/Safe-and-Stable-Ckpt2Safetensors-Conversion-Tool-GUI#downloads) для Windows/Linux.
исходная_ckpt = "/content/stable-diffusion-webui/models/Stable-diffusion/model.ckpt" #@param {type:"string"}
from google.colab import output
import os
from os import path

ckpt_name = path.basename(r'$исходная_ckpt')
sfname = path.splitext(исходная_ckpt)[0]

if not os.path.exists("/usr/local/lib/python3.8/dist-packages/safetensors"):
  !pip -q install safetensors
  output.clear()
else:
  None

if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pytorch_lightning"):
  !pip -q install pytorch-lightning
  output.clear()
else:
  None

if not os.path.exists("/usr/local/lib/python3.8/dist-packages/torchsde"):
  !pip -q install torchsde
  output.clear()
else:
  None

if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pandas"):
  !pip -q install pandas
  output.clear()
else:
  None

print("\033[01;38;05;136mКонвертирую...\033[")
import torch
from safetensors.torch import save_file
try:
  weights = torch.load(f"{исходная_ckpt}")["state_dict"]
  save_file(weights, f"{sfname}.safetensors")
  print("\033[01;38;05;34mГотово!")
except KeyError:
  print("модель содержит код NAI, с этим могут быть проблемы!\nконвертирую другим способом (протестируй потом работоспособность модели)")
  weights = torch.load(f"{исходная_ckpt}", map_location=torch.device('cpu'))
  save_file(weights, f"{sfname}.safetensors")
  print("\033[01;38;05;34mГотово!")
except RuntimeError:
  print("сконвертировать на GPU не получилось!\nконвертирую на CPU (протестируй потом работоспособность модели)")
  weights = torch.load(f"{исходная_ckpt}", map_location=torch.device('cpu'))["state_dict"]
  save_file(weights, f"{sfname}.safetensors")
  print("\033[01;38;05;34mГотово!")
except FileNotFoundError:
  print("указан неверный путь!")
except Exception as er:
  print(f"Ошибка: {er}")


#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Конвертор 🤗safetensors в 🥒ckpt (пакетный и одиночный режим){ display-mode: "form" }
#!pip -q install jax[cpu] flax accelerate transformers ftfy OmegaConf torch safetensors git+https://github.com/huggingface/diffusers
try:
    import jax 
except ImportError:
    !pip -q install jax[cpu]
try:
    import flax
except ImportError:
    !pip -q install flax
try:
    import accelerate
except ImportError:
    !pip -q install accelerate
try:
    import transformers
except ImportError:
    !pip -q install transformers
try:
    import ftfy
except ImportError:
    !pip -q install ftfy
try:
    import omegaconf
except ImportError:
    !pip -q install omegaconf
try:
    import torch
except ImportError:
    !pip -q install torch
try:
    import safetensors
except ImportError:
    !pip -q install safetensors
try:
    import diffusers
except ImportError:
    !pip -q install git+https://github.com/huggingface/diffusers
except AttributeError:
  print('требуется другая версия Pillow/PIL.\nна свой страх и риск, можешь переустановить:\n\033[1;36m!pip install Pillow==7.1.2\n!pip install PIL\n!pip install image\033[0m/nили скопировать ячейку в пустрой блокнот без GPU')
import os
from pathlib import Path
import numpy as np
try:
  from diffusers import FlaxStableDiffusionPipeline, StableDiffusionPipeline
except:
  !pip install Pillow==9.0.0
import torch
from safetensors.torch import save_file, load_file
safetensors = "/content/AbyssOrangeMix" #@param {type:"string"}
удалить_исходник = True #@param {type:"boolean"}
input_type = os.path.isfile(safetensors)
if input_type:
  #ckpt_name = Path(safetensors).stem + ".ckpt"
  ckpt_path = os.path.splitext(safetensors)[0]  + ".ckpt"
  weights = load_file(f"{safetensors}", device="cpu")
  torch.save(weights, ckpt_path)
  if удалить_исходник:
    !rm {safetensors}
  print("готово!")
if not input_type:
  print("пакетная конвертация")
  #%cd {safetensors}
  stfiles = os.listdir(safetensors)
  for f in stfiles:
    if f.lower().endswith('.safetensors'):
      inputsf = safetensors + "/" + f
      ckpt_path = os.path.splitext(inputsf)[0]  + ".ckpt"
      weights = load_file(f"{inputsf}", device="cpu")
      torch.save(weights, ckpt_path)
      print(f"сохранен: {ckpt_path}")
      if удалить_исходник:
        !rm {inputsf}
    continue
  print("готово!")
#@markdown Конвертор нужен, чтобы отпрунить модель, а safetensors пока не поддерживает прунинг. Берем большой сейфтензор, конвертируем в старый формат, <a href="#scrollTo=heHty0pzVYjw">пруним</a> по вкусу, <a href="#scrollTo=QuW09EK-mIvg">конвертируем обратно</a> в сейфтензорс

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title  сжатие Lora (пока результаты сомнительны) { display-mode: "form" }
#@markdown Можно указать путь до папки с лорами или одиночного файла<br>Перед выполнением будет показано предполагаемое время на задачу.<br>При выборе GPU-рантайма скорость прунинга уменьшается втрое. (должен быть включен до запуска блокнота в настройках <code>"Среда выполнения - Сменить среду выполнения")</code><br>Исходники можно удалить, заменяя их уменьшеной версией, а если не отметить опцию удаления - новые файлы будут лежать рядом с суффиксом _minify в имени.

источник = "/content/Lora" #@param {type:"string"}
удалить_исходник = True #@param {type:"boolean"}
runtime = "GPU" #@param ["CPU", "GPU"]
#установка
from google.colab import output
if not os.path.exists("/content/resize_lora"):
  print("Установка зависимостей...")
  !git clone https://github.com/PR0LAPSE/resize_lora /content/resize_lora
  !pip -q install einops
  !pip -q install -r /content/resize_lora/sd-scripts/requirements.txt
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/safetensors"):
    !pip -q install safetensors
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pytorch_lightning"):
    !pip -q install pytorch-lightning
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/torchsde"):
    !pip -q install torchsde
  if not os.path.exists("/usr/local/lib/python3.8/dist-packages/pandas"):
    !pip -q install pandas
  output.clear()

import os
from os import path
from pathlib import Path
from os import path
import enum
from subprocess import getoutput
import torch
from safetensors.torch import save_file
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#разбираемся с источником, перемещаем файлы не нуждающиеся в прунинге в отдельную папку
inputlora = источник
lorapath = Path(источник)
folderpath = os.path.dirname(inputlora) + "/"
suff = "_minify"
extention = os.path.splitext(inputlora)[-1]
outputname = folderpath + lorapath.stem + suff + extention
input_type = os.path.isfile(inputlora)
smallfiles = источник+"/not_need_prune/"
find = "find "+inputlora+" -size -10M -type f -print0 | xargs -0 -I {} mv {} "+smallfiles
#обработка одиночного файла
if input_type:
  print("\033[01;38;05;136mПрунинг лоры...")
  if runtime == "CPU":
    print("примерное время выполнения: 3 минуты\033[38;05;244m")
    !python /content/resize_lora/sd-scripts/resize_lora.py --device cpu --new_rank 8 --model {inputlora} --save_to {outputname}
  if runtime == "GPU":
    if "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in getoutput('nvidia-smi'):
      print("\033[3;31mблокнот же запущен без видеоускорителя! просчет будет производиться на процессоре!\033[0m")
      print("примерное время выполнения: 3 минуты\033[38;05;244m")
      !python /content/resize_lora/sd-scripts/resize_lora.py --device cpu --new_rank 8 --model {inputlora} --save_to {outputname}
    else:
      print("примерное время выполнения: 1 минута\033[38;05;244m")
      !python /content/resize_lora/sd-scripts/resize_lora.py --device cuda --new_rank 8 --model {inputlora} --save_to {outputname}
  print("\033[01;38;05;136mГотово")
  if удалить_исходник:
    !rm $inputlora
    os.rename(outputname, inputlora)
if not input_type:
#обработка папки
  print("пакетный прунинг ",len(os.listdir(inputlora))," файлов ...")
  for f in os.listdir(inputlora):
    if f.lower().endswith('.pt'):
      ptfile = inputlora + "/" + f
      ckptfile = inputlora + "/" + os.path.splitext(f)[0] + ".ckpt"
      os.rename(ptfile, ckptfile)
  for f in os.listdir(inputlora):
    if f.lower().endswith('.ckpt'):
      try:
        print('в наборе обнаружен pickle-tensor файл, конвертирую в safetensors...')
        weights = torch.load(f"{ckptfile}")["state_dict"]
        save_file(weights, f"{path.splitext(ckptfile)[0]}.safetensors")
      except RuntimeError:
        weights = torch.load(f"{ckptfile}", map_location=torch.device('cpu'))["state_dict"]
        save_file(weights, f"{path.splitext(ckptfile)[0]}.safetensors")
      except KeyError:
        weights = torch.load(f"{ckptfile}", map_location=torch.device('cpu'))
        save_file(weights, f"{path.splitext(ckptfile)[0]}.safetensors")
      except ValueError:
        print(f"ошибка конвертации в safetensors: файл поврежден или не является pickle-tensor форматом pt/ckpt")
      !rm $ckptfile
  if runtime == "CPU":
    timeconvert = len(os.listdir(inputlora)) * 3
    if timeconvert > 59:
      timeconvert = round(timeconvert / 60)
      print("\033[0mпримерное время выполнения:", timeconvert, "часов\033[38;05;244m")
    else:
      print("\033[0mпримерное время выполнения:", timeconvert, "минут\033[38;05;244m")
  else:
    if "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in getoutput('nvidia-smi'):
      print("\033[01;38;05;196mблокнот же запущен без видеоускорителя! просчет будет производиться на процессоре!")
      timeconvert = len(os.listdir(inputlora)) * 3
      print("\033[0mпримерное время выполнения:", timeconvert, "минут\033[38;05;244m")
    else:
      timeconvert = len(os.listdir(inputlora)) * 1
      print("\033[0mпримерное время выполнения:", timeconvert, "минут\033[38;05;244m")
  # перемещаем маленькие файлы в другую папку
  !mkdir $smallfiles 2>/dev/null
  !$find 2>/dev/null
  lorafiles = os.listdir(inputlora)
  for f in lorafiles:
    if f.lower().endswith('.ckpt') or f.lower().endswith('.safetensors'):
      inputfile = inputlora + "/" + f
      ext = os.path.splitext(inputfile)[-1]
      outfile = inputlora + "/" + os.path.splitext(f)[0] + suff + ext
      print(f"\033[01;38;05;68mВыполняется сжатие: \033[01;38;05;38m{f}\033[0m")
      if runtime == "CPU":
        !python /content/resize_lora/sd-scripts/resize_lora.py --device cpu --new_rank 8 --model {inputfile} --save_to {outfile}
        if удалить_исходник:
          !rm $inputfile
          os.rename(outfile, inputfile)
      else:
        if "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in getoutput('nvidia-smi'):
            !python /content/resize_lora/sd-scripts/resize_lora.py --device cpu --new_rank 8 --model {inputfile} --save_to {outfile}
            if удалить_исходник:
              !rm $inputfile
              os.rename(outfile, inputfile)
        else:
          !python /content/resize_lora/sd-scripts/resize_lora.py --device cuda --new_rank 8 --model {inputfile} --save_to {outfile}
          if удалить_исходник:
            !rm $inputfile
            os.rename(outfile, inputfile)
  try:
    if str(os.listdir(smallfiles)) == '[]':
      os.rmdir(smallfiles)
    else:
      # возвращаем маленькие файлы на место
      sfiles = os.listdir(smallfiles)
      for f in sfiles:
        sfile = smallfiles + "/" + f
        refund = inputlora + "/" + f
        os.replace(sfile, refund)
      os.rmdir(smallfiles)
  except Exception:
    None
  print("\033[01;38;05;46mГотово!")

In [ ]:
#@title Конвертер моделей pytorch .bin в .ckpt { display-mode: "form" }

#@markdown нужно указать путь до папки с файлами модели формата питорч, а не просто файла .bin!

#@markdown <details><summary><font color=red>как должно выглядить <u>содержимое</u> папки</font></summary><img src='https://dl.uploadgram.me/6373cae8820b1h?dl' height='500' /></details>

#@markdown [пример такой модели](https://huggingface.co/lambdalabs/sd-image-variations-diffusers/tree/main) - скачивать нужно всю папку, и вот такая папка указывается в параметрах, именно вся эта папка будет сконвертированна в один файлик модели с расширением .ckpt, пригодным для WebUI

#@markdown Если папку с моделью ты еще не скачал самостоятельно на гуглодиск, можешь вставить ссылку на репозиторий модели в поле ниже, тогда модель временно скачается в колаб и сконвертируется автоматически в ckpt куда ты укажешь, указывать дополнительно путь до нее будет не нужно, если указана ссылка на модель - поле пути до папки модели будет проигнорированно.

#@markdown Корректная ссылка должна выглядить только так: `https://huggingface.co/Юзер_Нейм/Модель_Нейм` а репозиторий должен быть открытым с доступом без всяких подтверждений лицензионных соглашений. В противном случае - скачайте папку вручную.

#@markdown Если будешь указывать путь до своей папки - ссылку удали из ее поля

ссылка_на_репозиторий_модели = "https://huggingface.co/IfanSnek/JohnDiffusion" #@param {type:"string"}

#@markdown  <hr>
!git clone {ссылка_на_репозиторий_модели}
import os
import os.path
from google.colab import output
from IPython.display import clear_output
папка_модели_питорч = "/content/JohnDiffusion" #@param {type:"string"}
куда_сохранить_ckpt = "/content/" #@param {type:"string"}
if not ссылка_на_репозиторий_модели or "huggingface.co" not in ссылка_на_репозиторий_модели:
  ckpt_name = os.path.basename(f"{папка_модели_питорч}")
  scrpath = "/content/.config/converter/ckptconverter.py"
  !curl -s -L --create-dirs -o {scrpath} https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/ckptconverter.py
  !python {scrpath} --model_path={папка_модели_питорч} --checkpoint_path={куда_сохранить_ckpt}/{ckpt_name}.ckpt
  if os.path.exists(куда_сохранить_ckpt+'/' + ckpt_name + '.ckpt'):
      output.clear()
      print("[1;32mВСЁ! модель в формате ckpt успешно создана.")
      print(куда_сохранить_ckpt+'/' + ckpt_name + '.ckpt')
      output.eval_js('new Audio("https://dl.uploadgram.me/6374ad94c98a5h?dl").play()')
  else:
      print("[1;31mчто-то пошло не так...")
      output.eval_js('new Audio("https://dl.uploadgram.me/6374ad9346cc1h?dl").play()')
else:
  !git lfs install
  output.clear()
  !cd /content
  output.clear()
  !git clone {ссылка_на_репозиторий_модели}
  output.clear()
  ckpt_name = '/'.join(ссылка_на_репозиторий_модели.split('/')[-1:])
  папка_модели_питорч = f"/content/{ckpt_name}"
  scrpath = "/content/.config/converter/ckptconverter.py"
  !curl -s -L --create-dirs -o {scrpath} https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/ckptconverter.py
  !python {scrpath} --model_path={папка_модели_питорч} --checkpoint_path={куда_сохранить_ckpt}/{ckpt_name}.ckpt
  if os.path.exists(куда_сохранить_ckpt+'/' + ckpt_name + '.ckpt'):
    !rm -rf /content/{ckpt_name}
    output.clear()
    print("[1;32mВСЁ! модель в формате ckpt успешно создана.")
    print(куда_сохранить_ckpt+'/' + ckpt_name + '.ckpt')
    output.eval_js('new Audio("https://dl.uploadgram.me/6374ad94c98a5h?dl").play()')
  else:
    !rm -rf /content/{ckpt_name}
    print("[1;31mчто-то пошло не так...")
    output.eval_js('new Audio("https://dl.uploadgram.me/6374ad9346cc1h?dl").play()')

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

#👈 Выгрузка моделей на HuggingFace

In [ ]:
#@title Выгрузка моделей/файлов до 5 Гб { display-mode: "form" }
HuggingFace_токен = "****************************************" #@param {type:"string"}
источник = "\u0444\u0430\u0439\u043B" #@param ["\u043F\u0430\u043F\u043A\u0430", "\u0444\u0430\u0439\u043B"]
что_загрузить = "/content/\u043A\u0430\u043A\u043E\u0439-\u0442\u043E/\u0444\u0430\u0439\u043B.safetensors" #@param {type:"string"}
репозиторий = "\u044E\u0437\u0435\u0440\u043D\u0435\u0439\u043C/\u0440\u0435\u043F\u043E\u0437\u0438\u0442\u043E\u0440\u0438\u0439" #@param {type:"string"}
#@markdown Токент нужно получить [здесь](https://huggingface.co/settings/tokens). При создании токена нужно указать **Role : Write**. <details><summary>скрин создания токена</summary><img src="https://dl.uploadgram.me/6373ee91cbbe5h?dl" height='300px'></details><br> В качестве источника можно указать путь до одного файла модели, или путь до папки с несколькими файлами - все содержимое папки будет отправленно в указанный репозиторий. В поле "репозиторий" нужно вписать твой юзернейм на HuggingFace и имя репозитория через слеш. Если репозитория еще нет - его <a href='https://huggingface.co/new'>нужно создать</a>, он должен быть публичным, чтобы загружаемые файлы потом были доступны по прямым ссылкам. Если в источнике будет файл недопустимого размера (больше 5 гигов) - загрузка даже не начнется, т.к. от дураков стоит проверка на размер файла, без этого случилось переполнение памяти и крах колаба. Для огромных файлов более 5гб нужно использовать ячейку ниже.  
import os
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/huggingface_hub"):
  from google.colab import output
  !pip install huggingface_hub==0.11.1
  output.clear()

if not HuggingFace_токен or len(HuggingFace_токен) < 36 or "hf_" not in HuggingFace_токен:
  print("Токен указан неверно. Получить токен можно тут: https://huggingface.co/settings/tokens")
else:
  try:
    from huggingface_hub import login
    login(f'{HuggingFace_токен}', add_to_git_credential=True)
  #except ImportError:
    #print('автоматический вход не удался, попробуй сейчас войти сам')
    #from huggingface_hub import notebook_login
    #notebook_login()
  except ValueError:
    print('\033[01;38;05;167mНеверный токен!')
  except Exception as er:
    print(f'\033[01;38;05;167mЧто-то пошло не так. Ошибка: {er}\033[0m \nПробуем прописать токен вручную...')
    if not os.path.exists('/root/.huggingface/token'):
      !mkdir /root/.huggingface/
      !touch /root/.huggingface/token
    tw = open("/root/.huggingface/token", "w+")
    tw.write(HuggingFace_токен)
    tw.close()

!sleep 1

import enum
class SIZE_UNIT(enum.Enum):
   BYTES = 1
   KB = 2
   MB = 3
   GB = 4
def convert_unit(size_in_bytes, unit):
   if unit == SIZE_UNIT.KB:
       return size_in_bytes/1024
   elif unit == SIZE_UNIT.MB:
       return size_in_bytes/(1024*1024)
   elif unit == SIZE_UNIT.GB:
       return size_in_bytes/(1024*1024*1024)
   else:
       return size_in_bytes
import os
def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
   size = os.path.getsize(file_name)
   return convert_unit(size, size_type)
file_path = что_загрузить
size = get_file_size(file_path, SIZE_UNIT.GB)
if size > 4.999:
  print('Размер модели : ', size ,  'Гб!\nЭто очень большой размер для данного метода загрузки.\nВоспользуйся методом в ячейке ниже, он предназначен для загрузки файлов более 5Гб.')
else:
  if not что_загрузить or not репозиторий:
    print("Не указан репозиторий или файл/папка для загрузки")
  if источник == "файл":
    from pathlib import Path
    p = Path(f"{что_загрузить}")
    print(p.name)
    print("жди, идет загрузка...")
    from huggingface_hub import HfApi
    api = HfApi()
    api.upload_file(
        path_or_fileobj=f"{что_загрузить}",
        path_in_repo=(p.name),
        repo_id=f"{репозиторий}",
        repo_type="model",
    )
  elif источник == "папка":
    print("жди, идет загрузка...")
    from huggingface_hub import HfApi
    api = HfApi()
    api.upload_folder(
        folder_path=f"{что_загрузить}",
        repo_id=f"{репозиторий}",
        repo_type="model",
    )
  if источник == "папка":
    print(f"загрузка завершена!\nтвои файлы тут: https://huggingface.co/{репозиторий}/tree/main \nвот прямые ссылки на модели:")
    from pathlib import Path
    dlfolder = Path(f"{что_загрузить}")
    for file_name in dlfolder.rglob('*.*'): 
      print(f"https://huggingface.co/{репозиторий}/resolve/main/{file_name.name}\n")
  if источник == "файл":
    filename = os.path.basename(f'{что_загрузить}')
    print(f"загрузка завершена!\nтвои файлы тут: https://huggingface.co/{репозиторий}/tree/main \nпрямая ссылка на модель: https://huggingface.co/{репозиторий}/resolve/main/{filename} ")

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Выгрузка файлов больше 5 гб { display-mode: "form" }
HuggingFace_токен = "\u0442\u0432\u043E\u0439 \u0442\u043E\u043A\u0435\u043D" #@param {type:"string"}
username = "\u044E\u0437\u0435\u0440\u043D\u0435\u0439\u043C"  #@param {type:"string"}
LFS_репозиторий = "\u0440\u0435\u043F\u043E\u0437\u0438\u0442\u043E\u0440\u0438\u0439" #@param {type:"string"}
что_выгружаем = "/content/models/my_biggest_model.safetensors" #@param {type:"string"}
#@markdown можно указать файл или целую папку с файлами. Если это папка - будет загружено все ее содержимое в репозиторий, но не сама папка, если не поставлена опция "сохранить структуру".
сохранить_структуру = False #@param {type:"boolean"}
#@markdown Копировать в папку репозитория или переместить? Перемещение быстрее, но исходные файлы с диска колаба удалятся, переместившись в папку репы. 
файлы_с_диска = "\u043F\u0435\u0440\u0435\u043C\u0435\u0449\u0430\u0435\u043C" #@param ["копируем", "перемещаем"]


import os
if not os.path.exists("/usr/local/lib/python3.8/dist-packages/huggingface_hub"):
  from google.colab import output
  !pip install huggingface_hub==0.11.1
  output.clear()
if not HuggingFace_токен or len(HuggingFace_токен) < 36 or "hf_" not in HuggingFace_токен:
  print("Токен указан неверно. Получить токен можно тут: https://huggingface.co/settings/tokens")
else:
  try:
    from huggingface_hub import login
    login(f'{HuggingFace_токен}', add_to_git_credential=True)
  #except ImportError:
    #print('автоматический вход не удался, попробуй сейчас войти сам')
    #from huggingface_hub import notebook_login
    #notebook_login()
  except ValueError:
    print('\033[01;38;05;167mНеверный токен!')
  except Exception as er:
    print(f'\033[01;38;05;167mЧто-то пошло не так. Ошибка: {er}\033[0m \nПробуем прописать токен вручную...')
    if not os.path.exists('/root/.huggingface/token'):
      !mkdir /root/.huggingface/
      !touch /root/.huggingface/token
    tw = open("/root/.huggingface/token", "w+")
    tw.write(HuggingFace_токен)
    tw.close()

!sleep 1

!git lfs install --skip-smudge

!git -c filter.lfs.smudge= clone https://huggingface.co/{username}/{LFS_репозиторий}

if сохранить_структуру:
  if файлы_с_диска == "копируем":
    !cp -f -i -R {что_выгружаем} /content/{LFS_репозиторий}
  elif файлы_с_диска == "перемещаем":
    !mv -f -i {что_выгружаем} /content/{LFS_репозиторий}
if not сохранить_структуру:
  if файлы_с_диска == "копируем":
    !cp -f -i -R -t /content/{LFS_репозиторий} {что_выгружаем}
  elif файлы_с_диска == "перемещаем":
    !mv -f -i -t /content/{LFS_репозиторий} {что_выгружаем}

from huggingface_hub import Repository
repo = Repository(local_dir=f"/content/{LFS_репозиторий}")
!huggingface-cli lfs-enable-largefiles /content/{LFS_репозиторий}

!sleep 2

#запустить выгрузку
repo.push_to_hub(commit_message="upload with huggingface-cli")
!sleep 1
print(f"загрузка завершена!\nтвои файлы тут: https://huggingfce.co/{username}/{LFS_репозиторий}/tree/main")

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

# 👈 Выгрузка файлов на файлообменники

In [ ]:
#@title Загрузка на файлообменники типа anonfiles до 20 Гб на 1 файл
import requests
Cервер = "filechan.org" #@param ["anonfiles.com", "openload.cc", "bayfiles.com", "lolabits.se", "vshare.is", "upvid.cc", "hotfile.io", "rapidshare.nu", "filechan.org", "letsupload.cc", "share-online.is", "megaupload.nz", "myfile.is"]
url = f'https://api.{Cервер}/upload'
Файл = "/content/gdrive/MyDrive/my_willy.7z" # @param {type: "string"}
x = requests.post(url, files = {'file': open(Файл,'rb')},)
print("\033[01;38;05;46m Твоя ссылка: " + x.json()["data"]["file"]["url"]["full"])
#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Выгрузка на разные файлообменники { display-mode: "form" }
import os
import subprocess
import json
try:
  if not os.path.exists('/content/.config/go-ul_linux_x64'):
    !wget -P /content/.config https://github.com/bunnykek/go-upload/releases/download/tag/go-ul_linux_x64
    !chmod 755 /content/.config/go-ul_linux_x64
    from google.colab import output
    output.clear()
  else:
    !chmod 755 /content/.config/go-ul_linux_x64
    output.clear()
except Exception as er:
  print('Ошибка: ', er)
!sleep 1
файлообменник = "pixeldrain" #@param ['catbox', 'fileio', 'filemail', 'gofile', 'krakenfiles', 'letsupload', 'mixdrop', 'pixeldrain', 'racaty', 'transfersh', 'uguu', 'wetransfer', 'workupload', 'zippyshare']
файл_для_выгрузки = "/content/output.zip" #@param {type:"string"}
filehost, infile = файлообменник, файл_для_выгрузки
message = "Файл: {0}\nСсылка: {2}"
cwd = '/content/.config/'
def upload(filehost:str, filePath: str):
    subprocess.Popen(["./go-ul_linux_x64", filehost,
                     '-f', infile, '-j', 'response.json'], 
                     cwd=cwd).wait()
    response = json.load(open(cwd+'response.json'))
    if not response['jobs'][-1]['ok']:
        print(f"{response['jobs'][-1]['error_text']}")
    else:
        print(message.format(response['jobs'][-1]['filename'], filehost, response['jobs'][-1]['url']))
upload(filehost, infile)
#@markdown обменник       | лимит  |ǀ|   обменник   | лимит
#@markdown ---------------|--------||---------------|---------------
#@markdown Catbox         | 200 MB |ǀ|  pixeldrain  | 10 GB 
#@markdown file.io        | 2 GB   |ǀ|  Racaty      | 10 GB 
#@markdown Filemail       | 5 GB   |ǀ|  transfer.sh | unlim
#@markdown Gofile         | unlim  |ǀ|  Uguu        | 128 MB
#@markdown KrakenFiles    | 1 GB   |ǀ|  WeTransfer  | 2 GB 
#@markdown LetsUpload.io  | 10 GB  |ǀ|  workupload  | 2 GB
#@markdown MixDrop        | unlim  |ǀ|  zippyshare  | 500 MB 

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

# 👈 Распаковка и запаковка архивов разных форматов

In [ ]:
#@title Распаковка архивов { display-mode: "form" }
import os
import pathlib
from pathlib import Path 
файл_архива = "/content/xenopus_laevis.rar"  # @param {type:"string"}
куда_распаковать = "/content/gdrive/MyDrive/"  # @param {type:"string"}
создать_подпапку = True #@param {type:"boolean"}
#@markdown Если папка не существует - она будет создана, а если не указать место распаковки, архив распакуется в папку /content/распаковка
пароль = "" #@param {type:"string"}
subfolder = os.path.basename(файл_архива).split('.')[0]
arctype = pathlib.Path(f'{файл_архива}').suffix
if пароль:
  passADD = f'-P {пароль}'
else:
  passADD = ''
if not куда_распаковать:
  куда_распаковать = "/content/распаковка"
if создать_подпапку:
  extract_path = куда_распаковать + "/" + subfolder
if not создать_подпапку:
  extract_path = куда_распаковать
def raspakovka():
  if "rar" in arctype or "RAR" in arctype:
    os.makedirs(extract_path, exist_ok=True)
    !unrar -y x "{файл_архива}" "{extract_path}" {passADD} -o
  else:
    os.makedirs(extract_path, exist_ok=True)
    !7z -y -bt x "{файл_архива}" -o"{extract_path}" {passADD}
if "liz" in arctype or "LIZ" in arctype or "zst" in arctype or "ZST" in arctype or "ace" in arctype or "ACE" in arctype:
    print("\033[01;38;05;167mТип архива \033[01;38;05;178m" + arctype + "\033[01;38;05;167m не поддерживается!\033[39m")
else:
  raspakovka()
#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

In [ ]:
#@title Создать архив
import os
from pathlib import Path
имя_архива = "sdwui"  # @param {type:"string"}
что_архивировать = "/content/stable-diffusion-webui"  # @param {type:"string"}
пароль = "" #@param {type:"string"}
#@markdown пароль можно не заполнять.<br><small>готовый файл архива будет лежать рядом с иходной папкой/файлом<small>
сжатие = "\u0431\u044B\u0441\u0442\u0440\u043E (zip)" #@param ["быстро (zip)", "хорошо (7z)"]
archpath = Path(что_архивировать)
dirpath = str(archpath.parent) + "/"
if not os.path.exists(что_архивировать):
  print("Источник указан неверно, архивировать нечего...")
else:
  if сжатие == "быстро (zip)":
    if пароль:
      !7z a -p{пароль} -tzip -mx=4 "{имя_архива}.zip" {что_архивировать}
    else:
      !7z a -tzip -mx=4 "{имя_архива}.zip" {что_архивировать}
    print(f"\033[01;38;05;178mВот полный путь до архива:\033[01;38;05;110m {dirpath}{имя_архива}.7z\033[39m")
  elif сжатие == "хорошо (7z)":
    if пароль:
      !7z a -mhe=on -p{пароль} -t7z -m0=lzma2 -mx=9 -mfb=64 -md=1024m -ms=on "{имя_архива}.7z" {что_архивировать}
    else:
      !7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=1024m -ms=on "{имя_архива}.7z" {что_архивировать}
    print(f"\033[01;38;05;178mВот полный путь до архива:\033[01;38;05;110m {dirpath}{имя_архива}.7z\033[39m")

#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

# 👈 Операции с файлами/папками

In [ ]:
#@title копирование, перемещение, создание символьных ссылок, очистка папок, удаление
import os
import pathlib
from pathlib import Path
откуда = "/content/a" #@param {type:"string"}
куда = "/content/b/c" #@param {type:"string"}
что_делать = "\u0441\u043E\u0437\u0434\u0430\u0442\u044C \u0441\u0438\u043C\u043B\u0438\u043D\u043A\u0438 \u043D\u0430 \u0432\u0441\u0435 \u0441\u043E\u0434\u0435\u0440\u0436\u0438\u043C\u043E\u0435 \u043F\u0430\u043F\u043A\u0438 \u0438\u043B\u0438 \u0444\u0430\u0439\u043B \u0432 \u0434\u0440\u0443\u0433\u043E\u0439 \u043F\u0430\u043F\u043A\u0435" #@param ["копировать только содержимое папки или файл", "копировать файл или папку с ее содержимым в другую папку", "переместить только содержимое папки или файл в другую папку", "переместить папку или файл в другую папку", "создать симлинки на все содержимое папки или файл в другой папке", "создать симлинк на папку или файл в другой папке", "удалить содержимое папки или файл", "удалить папку или файл"]
#@markdown Важно задать источник и тип операции, а если выходная папка не существует - она будет создана, если это возможно. Файлы могут быть перезаписаны, если существуют в месте назначения!
if что_делать == "копировать только содержимое папки или файл":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
      if not os.path.exists(куда):
        if not os.path.isdir(куда):
            os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
        print("\033[1;34mкопирование содержимого папки...\033[0m")
        !cp -v -H -r "{откуда}"/* -t "{куда}"
        print("\033[1;32mкопирование завершено\033[0m")
    else:
      if os.path.isfile(куда):
        print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      else:
        print("\033[1;34mкопирование файла...\033[0m")
        if not os.path.exists(куда):
          if not os.path.isdir(куда):
            os.makedirs(куда)
        !cp -H -v -r "{откуда}" -t "{куда}"
        print(f"\033[1;32mфайл \033[1;36m{откуда}\033[1;32m успешно скопирован в \033[1;36m{куда}\033[0m")
elif что_делать == "копировать файл или папку с ее содержимым в другую папку":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
          if not os.path.exists(куда):
            if not os.path.isdir(куда):
              os.makedirs(куда)
            else:
              print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
    print("\033[1;34mкопирование...\033[0m")
    !cp -v -r "{откуда}" "{куда}"
    print("\033[1;32mкопирование завершено\033[0m")
elif что_делать == "переместить только содержимое папки или файл в другую папку":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
      if not os.path.exists(куда):
        if not os.path.isdir(куда):
          os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mперемещение содержимого папки...\033[0m")
      !mv -v "{откуда}"/* "{куда}"
      print("\033[1;32mперемещение завершено\033[0m")
    else:
      if not os.path.isdir(откуда):
        if not os.path.exists(куда):
          if not os.path.isdir(куда):
            os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mперемещение файла...\033[0m")
      !mv -v "{откуда}" "{куда}"
      print("\033[1;32mперемещение завершено\033[0m")
elif что_делать == "переместить папку или файл в другую папку":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
      if not os.path.exists(куда):
        if not os.path.isdir(куда):
          os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mперемещение содержимого папки...\033[0m")
      !mv -v "{откуда}" "{куда}"
      print("\033[1;32mперемещение завершено\033[0m")
    else:
      if not os.path.isdir(откуда):
        if not os.path.exists(куда):
          if not os.path.isdir(куда):
            os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mперемещение файла...\033[0m")
      !mv -v "{откуда}" "{куда}"
      print("\033[1;32mперемещение завершено\033[0m")
elif что_делать == "создать симлинки на все содержимое папки или файл в другой папке":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует")
  else:
    if os.path.isdir(откуда):
      if not os.path.exists(куда):
        if not os.path.isdir(куда):
          os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mсоздание символических ссылок на содержимое папки...\033[0m")
      !ln -sf -v "{откуда}"/* "{куда}"
      print("\033[1;32mсимвольные ссылки созданы\033[0m")
    else:
      if not os.path.isdir(откуда):
        if not os.path.exists(куда):
          if not os.path.isdir(куда):
            os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mсоздание символической ссылки на файл...\033[0m")
      !ln -sf -v "{откуда}" "{куда}"
      print("\033[1;32mсимвольная ссылка на файл создана в указанной папке\033[0m")
elif что_делать == "создать симлинк на папку или файл в другой папке":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
      if not os.path.exists(куда):
        if not os.path.isdir(куда):
          os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mсоздание символической ссылки на папку...\033[0m")
      !ln -sf -v "{откуда}" "{куда}"
      print("\033[1;32mсимвольная ссылка создана\033[0m")
    else:
      if not os.path.isdir(откуда):
        if not os.path.exists(куда):
          if not os.path.isdir(куда):
            os.makedirs(куда)
        else:
          print("\033[1;31mзадай другое имя выходной папки!\n\033[01;38;05;208mимеется файл с точно таким же именем, что не допустимо в линукс!\033[0m")
      print("\033[1;34mсоздание символической ссылки на файл...\033[0m")
      !ln -sf -v "{откуда}" "{куда}"
      print("\033[1;32mсимвольная ссылка на файл создана в указанной папке\033[0m")
elif что_делать == "удалить содержимое папки или файл":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    if os.path.isdir(откуда):
      print("\033[1;34mУдаление...\033[0m")
      !rm -rfv "{откуда}"/* "{куда}"
      print("\033[1;32mвсе файлы в папке удалены\033[0m")
    else:
      if not os.path.isdir(откуда):
        print("\033[1;34mУдаление файла...\033[0m")
        !rm -rfv "{откуда}" "{куда}"
        print("\033[1;32mфайл удален\033[0m")
elif что_делать == "удалить папку или файл":
  if not os.path.exists(откуда):
    print("\033[1;31mисточник указан не верно - он не существует\033[0m")
  else:
    print("\033[1;34mУдаление...\033[0m")
    !rm -rfv "{откуда}" "{куда}"
    print("\033[1;32mудаление завершено\033[0m")
#@markdown <center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>

# Другое



In [ ]:
#@title запустить Терминал Linux { display-mode: "form" }
from IPython.display import JSON, HTML, IFrame
from google.colab import output
from subprocess import getoutput
import os
import urllib.request
html_code_url = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/terminal.html"
def _run_once(f):
    def wrapper(*args, **kwargs):
        if not wrapper.has_run:
            wrapper.has_run = True
            return f(*args, **kwargs)
    wrapper.has_run = False
    return wrapper
@_run_once
def __get_html_code(html_code_url=html_code_url):
    fp = urllib.request.urlopen(html_code_url)
    mybytes = fp.read()
    html_code = mybytes.decode("utf8")
    fp.close()
    return html_code
def __shell(command):
    if command.startswith("cd"):
        path = command.strip().split(maxsplit=1)[1]
        os.chdir(path)
        return JSON([""])
    return JSON([getoutput(command)])
output.register_callback("shell", __shell)
def html(height=400, html_code=__get_html_code()):
    html_code = html_code.replace("$$height$$", str(height))
    return HTML(html_code)
html()

#@markdown <hr><center> <a href="#scrollTo=gnzJeBa5OYO3">🔝</a>&emsp;<a href="#scrollTo=R-xAdMA5wxXd">🚀</a></center>
#@markdown <details><summary>быстрые ссылки</summary><ul><li><a href="#scrollTo=10iWu8h1UTvQ">установка и запуск</a></li><li><a href="#scrollTo=MTex_2fJwYjn">выбор и установка нескольких моделей</a></li><li><a href="#scrollTo=R-xAdMA5wxXd">запуск/перезапуск</a></li><li><a href="#scrollTo=V-2LlLHZPgoB">скачивание архива с картинками</a></li><li><a href="#scrollTo=w_29ofOEGng_">подключение/переподключение/сброс кэша гуглодиска</a></li><li><a href="#scrollTo=TuwHDqutjgZd">массовая загрузка по прямым ссылкам</a></li><li><a href="#scrollTo=62QDyjXtYHjF">загрузчик моделей с civitai.com</a></li><li><a href="#scrollTo=VirGjpcNCCKX">скачивание с торрентов</a></li><li><a href="#scrollTo=heHty0pzVYjw">прунинг и конвертация моделей</a></li><li><a href="#scrollTo=j7N-__N4f9uY">выгрузка моделей на HuggingFace</a></li><li><a href="#scrollTo=zVmDZeIAIcja">выгрузка на файлообменники</a></li><li><a href="#scrollTo=k98WImeXH5pK">запаковка/распаковка архивов</a></li><li><a href="#scrollTo=Oh37kTko2shA">операции с папками/файлами/симлинками</a></li></ul></details>